In [ ]:
pip install -q yahoo_fin

In [ ]:
!mkdir -p data results logs


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [ ]:
pip install  git+https://github.com/ahsank/runml@testv2.01#egg=runml

In [ ]:
import runml.pipeline
from runml import pipeline,findata


In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

In [ ]:
findata.EPOCHS=200
new = []
model = "gstock-4b-test"
tickers = [ 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX', 'DBOEY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']


In [ ]:
findata.EPOCHS=200
model = "val-3a-test"
new = ['DXC', 'FSLR', 'STLD', 'FICO', 'SRE', 'BKR', 'CTAS', 'PEP', 'ADI', 'ROP' ]
tickers = ['AAL', 'ADI', 'ALB', 'ANF', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BKR', 'BGFV', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL', 'PXD',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TCN', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

In [ ]:
findata.EPOCHS=200
model = "cry-3a-test"
tickers = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']

In [ ]:
findata.EPOCHS=200
model = "etf-3a-test"
tickers = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]

In [ ]:
findata.EPOCHS=200
model = "mcap-3b-test"
tickers = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS', 'DELL',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']

In [ ]:
pipeline.IS_VERBOSE = False
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


df  = pipeline.runModelCombinedVola(tickers, model, mod, False, loss=lossfn)


In [ ]:
addAllocHL(df, 0.10, 1)
df1 = df #df[df.Gain > 0]
df1.sort_values('Alloc', ascending=False).head(50)
# df1.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index]
#df.sort_values('Alloc', ascending=False).head(50)


,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
16,DIS,80.62,0.93,658.70,614.46,112.73,121.180000,0.07,0.48,-99.43,66.63,124.800003,0.11,0.66,386.06,387.62,108.919998,-0.03,0.07,8.300000
21,INTC,24.81,0.91,234.13,234.92,31.88,35.320000,0.11,0.56,56.35,84.35,35.599998,0.12,0.71,202.68,217.81,34.049999,0.07,0.11,8.281818
12,COST,283.35,0.93,2605.73,1803.03,729.18,682.750000,-0.06,0.70,1653.44,9.77,744.570007,0.02,0.58,1469.74,-60.23,666.070007,-0.09,-0.06,8.133333
39,UPS,81.80,0.90,835.60,688.84,147.59,158.669998,0.08,0.58,336.11,-4.47,158.729996,0.08,0.69,689.98,598.45,141.910004,-0.04,0.08,7.750000
8,BRK-B,173.00,0.93,1140.50,735.22,402.10,379.760010,-0.06,0.66,672.96,-4.98,411.660004,0.02,0.56,906.96,-21.05,389.630005,-0.03,-0.03,6.966667
6,BA,118.72,0.91,1566.62,1128.82,167.22,151.759995,-0.09,0.47,16.60,49.02,178.270004,0.07,0.77,1192.70,1441.37,161.059998,-0.04,-0.04,6.850000
37,TXN,99.83,0.90,827.92,526.94,177.48,170.139999,-0.04,0.67,401.30,189.99,179.720001,0.01,0.78,897.20,665.01,167.139999,-0.06,-0.04,6.500000
29,NVDA,88.18,0.88,3530.26,1747.79,877.35,781.890015,-0.11,0.60,1766.96,-533.36,919.039978,0.05,0.73,2336.60,-6.80,834.510010,-0.05,-0.05,6.400000
17,DELL,21.99,0.78,366.10,160.82,125.06,107.029999,-0.14,0.62,191.35,-29.14,129.330002,0.03,0.56,180.70,-162.25,111.839996,-0.11,-0.11,5.800000
40,WMT,37.59,0.88,193.55,91.48,60.16,57.490002,-0.04,0.63,85.03,46.21,60.570000,0.01,0.83,178.08,105.59,57.930000,-0.04,-0.04,5.800000


In [ ]:
findata.EPOCHS=200
model = "vols-4b-test"
pending = ['ASTR', 'SMWB']
next = []
# exclude = ['GTSWF', 'CBNT', 'CCL', 'CREX'. 'DTEA'. 'ILAL',
# 'TNXP', 'MRIN', 'MULN', 'AHT',
# 'LILM', 'JMIA', 'RKLB', 'VRAR', 'WISH',]
tickers = [ 'AIRI', 'AEXAF', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS', 'AUGX', 'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CLOV', 'CRCT', 'CXM', 'CZOO',
            'DLO',  'DM', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MRDB', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PTON', 'REAL', 'REI', 'RIG', 'RKLB', 'RUM', 'RVYL',
            'SABR', 'SAVE', 'SFIX', 'SLE', 'SMWB', 'SOUN', 'SST', 'STEM',
            'TDW', 'TRUP', 'TELL',
            'VTEX', 'YEXT', 'WKME']
future = []

In [ ]:
pending = ['PAR']
findata.EPOCHS=200
model = "ipos-4a-test"
next = []
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP', 'AYX',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HIMS','HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JKS', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL','MTCH',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PERI', 'PANW',  'PAYX', 'PD', 'PLUG', 'PI', 'PINS', 'PUBM',
           'RBLX', 'RMBS', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SPLK', 'SQ', 'SPOT',
           'TEAM', 'TDOC', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']

In [ ]:
findata.EPOCHS=200
model = "cetf-3b-test"
tickers = ['ARGT', 'EWD', 'FLN', 'FM',
           'ECH', 'EPHE', 'EWA', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'THD', 'TUR', 'VNM', 'XCEM']

In [ ]:
findata.EPOCHS=200
model = "com-4a-test"
tickers = [ 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']

In [ ]:
lossfn = "huber_loss"
pipeline.IS_VERBOSE = False
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
df  = pipeline.runModelCombinedVola(tickers, model, mod, True, loss=lossfn)

In [ ]:
addAllocHL(df, 0.10, 1)
# df1 = df[df.Gain > 0]
df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
# df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
61,THQQF,2.08,0.76,81.15,132.93,2.50,3.410000,0.36,0.65,87.56,70.49,4.440000,0.78,0.56,29.14,74.70,2.600000,0.04,0.36,6.933333
2,ADYEY,7.06,0.89,156.03,140.34,12.57,15.520000,0.23,0.73,110.25,133.19,16.389999,0.30,0.71,121.61,106.99,13.130000,0.04,0.23,8.421739
55,SE,41.05,0.88,1467.92,1374.32,62.86,75.650002,0.20,0.74,1519.28,1025.70,76.779999,0.22,0.68,1199.54,1114.34,58.970001,-0.06,0.20,8.200000
20,ERJ,4.54,0.86,125.83,81.48,25.99,21.959999,-0.16,0.78,155.53,88.84,25.040001,-0.04,0.73,120.72,44.67,20.610001,-0.21,-0.16,7.725000
52,SFTBY,17.46,0.89,179.12,161.35,24.98,28.230000,0.13,0.68,160.06,139.70,30.290001,0.21,0.76,160.72,159.27,24.670000,-0.01,0.13,8.053846
45,OTGLY,4.46,0.84,68.70,120.15,7.16,8.080000,0.13,0.72,78.01,97.56,8.230000,0.15,0.77,67.07,122.93,6.450000,-0.10,0.13,7.169231
3,BAK,6.49,0.92,102.26,112.54,8.94,9.850000,0.10,0.70,110.68,132.43,10.310000,0.15,0.78,99.82,113.38,7.900000,-0.12,0.10,8.400000
47,POAHF,45.25,0.86,412.93,416.67,54.23,60.290001,0.11,0.74,398.99,435.77,64.209999,0.18,0.78,464.24,482.12,52.369999,-0.03,0.11,7.327273
26,GLOB,113.82,0.84,1497.28,1870.31,180.83,206.690002,0.14,0.75,1588.29,1523.15,200.279999,0.11,0.80,1776.24,1770.04,173.550003,-0.04,0.11,6.945455
37,LYB,42.38,0.91,425.11,297.86,102.13,88.790001,-0.13,0.79,461.28,365.25,100.389999,-0.02,0.70,462.99,304.38,92.470001,-0.09,-0.09,8.100000


In [ ]:
pip install seaborn

In [ ]:
import seaborn as sns

Todos:



1.   Load previously saved model before training
2.   Allow eval only without test/train spliting
3.   Incremental data load
4.    Add S&P, QQQ etc to the model
5. Save model to google drive






In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !cp  /content/drive/MyDrive/colab/results/* ./results/
else:
  ! cp ~/Google\ Drive/My\ Drive/colab/results/* ./results/

Run following in local Jupyter

Run following 2 cells for colab to import libraries

Run following if the library code is modified locally

In [ ]:
!git clone https://github.com/ahsank/runml

In [ ]:
import os
import sys
path = os.path.expanduser('~/src/runml')
print(path)
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata

/Users/ahsank/src/runml


In [ ]:
import importlib
import sys
path = 'runml'
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata


In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
from importlib import reload
reload(findata)
reload(pipeline)

<module 'runml.pipeline' from '/Users/ahsank/src/runml/runml/pipeline.py'>

In [ ]:
import pandas as pd
df1 = pipeline.fetch_data('ABNB')
#df1.sort_index(inplace=True)
df1 = df1.round(3)

df2 = pd.read_csv('data/ABNB_2023-04-15.csv', index_col=0)
df2.index =  pd.to_datetime(df2.index)
# df2 = df2.round(3)
#data2 = data.tail(1000)
#pd.concat([df1,df2]).drop_duplicates(keep=False)
#df1.compare(df2)
#df2.index.dtype
df1

In [ ]:
display(df.sort_values('Gain', ascending=False))
display(df[["Buy", "Sell", "Total"]].sum()/len(df))


In [ ]:
results['ABNB'].pdata.__dict__.keys()

In [ ]:
findata.EPOCHS=10
tickers1 = ['ARKK', 'ARKW', 'DAPP', 'DTEC', 'EEM', 'FPX',
            'GBTC', 'GLD',
            'ICLN', 'IJR', 'IPO', 'IPOS', 'IWM',
            'JETS', 'KEMQ',
            'MGK', 'MGV', 'MOAT', 'MTUM',
            'QQQ', 'SLV','SMH', 'SMOG',
            'SPY', 'VNQ', 'VT', 'VTI', 'WDIV',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME' ]
mod = pipeline.RateReturnOnly(pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap()]))
df1 = pipeline.runModelCombined(tickers1, 'etf2b', mod, True)

In [ ]:
df1.sort_values('Gain', ascending=False)

In [ ]:
findata.EPOCHS=200
tickers3 = [
           'AHT', 'AMSC', 'ASTR', 'ATOM',
           'BKKT', 'BGFV', 'BGSF', 'CBNT', 'CLOV',
           'DNMR', 'ERJ', 'EVGO',
           'FSLY',  'FTCH', 'GOGO', 'HIVE',
           'ILAL', 'INLX', 'JMIA', 'JOBY',  'KULR', 'MTTR',
           'MYTE', 'NEPH', 'ONDS', 'MQ',
           'PETS', 'PTON', 'SFIX', 'SFT', 'STNE',
           'ULH', 'VRAR', 'WISH']
lossfn = "huber_loss"
# lossfn = "mean_squared_error"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df3, results = pipeline.runModelCombined(tickers3, 'vols', mod, False, loss=lossfn)

In [ ]:
df3

In [ ]:
import pandas as pd
seq = list()
for ticker, result in results.items():
    tempdf = result.final_df
    tempdf['predicted_rate'] = (tempdf['adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    tempdf['true_rate'] = (tempdf['true_adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    seq.append(tempdf[['ticker', 'date', "predicted_rate", 'true_rate']])

detailstat = pd.concat(seq)



In [ ]:
normalstat = detailstat[detailstat['predicted_rate'].between(-2,2) & detailstat['true_rate'].between(-2,2)]
largestat = normalstat[~ normalstat['predicted_rate'].between(-0.25,0.25) & ~ normalstat['true_rate'].between(-0.25,0.25)]
largestat.sample(10)

In [ ]:
ax = largestat.plot.scatter(x='predicted_rate', y='true_rate',c='DarkBlue', figsize=(10,10))
# ax.set_yscale('log')
# ax.set_xscale('log')

In [ ]:
# import matplotlib.pyplot as plt

# plt.hist([normalstat['predicted_rate'],normalstat['predicted_rate']],
#          bins=100, range=(-1,1), color = ['r','g'])

corr = largestat[["predicted_rate", "true_rate"]].corr()

corr.style.background_gradient(cmap='coolwarm')

In [ ]:

sns.regplot(x=largestat["predicted_rate"], y=largestat["true_rate"])


In [ ]:
sns.heatmap(normalstat[["predicted_rate", "true_rate"]].corr(), annot = True, fmt='.2g',cmap= 'coolwarm')

In [ ]:
!rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/


sending incremental file list
gstock-4b-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
gstock-4b-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
gstock-4b-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 9,649,178 bytes  received 73 bytes  6,432,834.00 bytes/sec
total size is 215,232,944  speedup is 22.31
